In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.uqat.ca/etudes/ecole-de-genie/baccalaureat-en-genie-mecanique/"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts for webdriver to click on

In [10]:
driver.find_elements_by_class_name("btn-accept")[0].click()

In [12]:
driver.find_elements_by_link_text("Plan de formation")[1].click()

IndexError: list index out of range

need to manually click on "Plan de formation" button!!

In [13]:
page_soup = soup(driver.page_source, 'lxml')

In [18]:
links = page_soup.find("div", {"class": "col-xs-12"}).findAll("a")
len(links)

61

In [19]:
link_texts = [link.text.strip() for link in links]
link_texts

['GEN0009 - Atelier : santé et sécurité au travail (1 cr.)',
 'GEN1005 - Calcul I (2 cr.)',
 "GEN1100 - Introduction au génie et aux projets d'ingénierie (2 cr.)",
 'GEN2210 - Statique (2 cr.)',
 'GEN2250 - Communication graphique (2 cr.)',
 'GEN3307 - Informatique I (3 cr.)',
 'GEN4110 - Chimie pour ingénieurs (2 cr.)',
 'GEN1007 - Calcul II (2 cr.)',
 "GEN1201 - L'ingénieur et la société II (1 cr.)",
 'GEN2800 - Morphologie des machines (3 cr.)',
 'GEN3115 - Dessin et conception assistés par ordinateur (2 cr.)',
 'GEN4139 - Techniques de mesure en génie mécanique (2 cr.)',
 'GEN5110 - Matériaux (2 cr.)',
 'MEC1400 - Résistance des matériaux I (3 cr.)',
 'GEN1004 - Équations différentielles ordinaires et aux dérivées partielles (3 cr.)',
 'GEN3250 - Probabilités et statistiques (3 cr.)',
 'GEN3420 - Logiciels de simulation (3 cr.)',
 'GEN4129 - Fabrication mécanique (3 cr.)',
 'MEC2209 - Thermodynamique (3 cr.)',
 'GEN2813 - Laboratoire mécanique I (1 cr.)',
 "GEN4230 - Éléments d'éle

# 2. Test run - try to scrape first course

In [20]:
driver.find_element_by_link_text(link_texts[0]).click()

In [21]:
driver.switch_to.window(driver.window_handles[1])

In [22]:
driver.current_url

'https://www.uqat.ca/etudes/cours/?code=GEN0009'

In [23]:
page_soup = soup(driver.page_source, 'lxml')

In [25]:
course_title = page_soup.find("h1", {"id": "gesta-cours-titre"}).text.strip()
course_title

'GEN0009 - Atelier : santé et sécurité au travail'

In [26]:
course_code = course_title.split(" - ")[0]
course_code

'GEN0009'

In [27]:
course_name = course_title.split(" - ")[1]
course_name

'Atelier : santé et sécurité au travail'

In [31]:
try:
    course_desc_objective = page_soup.find("p", {"id": "gesta-cours-objectifs"}).text.strip()
except AttributeError:
    course_desc_objective = ""
    print("No objective")
    
try:
    course_desc_content = page_soup.find("p", {"id": "gesta-cours-contenu"}).text.strip()
except AttributeError:
    course_desc_content = ""
    print("No content")

if course_desc_content == "Chargement...":
    course_desc = course_desc_objective
else:
    course_desc = course_desc_objective + " " + course_desc_content
    
course_desc

"Identifier les principales lois, normes et organismes reliés à la SST. Porter un jugement critique sur les structures en place. Contribuer à l'application des principales lois, règlements et normes dans son milieu de travail. Sensibiliser à l'impact des accidents de travail et à l'importance d'une législation dans ce domaine. Développer une approche préventive."

In [33]:
driver.close()

In [34]:
driver.switch_to.window(driver.window_handles[0])

In [35]:
driver.current_url

'https://www.uqat.ca/etudes/ecole-de-genie/baccalaureat-en-genie-mecanique/'

# 3. Test run complete. Implement automation script to scrape all courses

In [36]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

course_codes = []
course_names = []
course_descs = []
no_window_opened = False
counter = 0

for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    link.click()
        
    time.sleep(3)
    
    #switch tabs if needed
    if len(driver.window_handles) == 2:
        driver.switch_to.window(driver.window_handles[1])
    else:
        no_window_opened = True
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title = page_soup.find("h1", {"id": "gesta-cours-titre"}).text.strip()

    course_code = course_title.split(" - ")[0]
    course_name = course_title.split(" - ")[1]
    try:
        course_desc_objective = page_soup.find("p", {"id": "gesta-cours-objectifs"}).text.strip()
    except AttributeError:
        course_desc_objective = ""
        print("No objective")
        
    try:
        course_desc_content = page_soup.find("p", {"id": "gesta-cours-contenu"}).text.strip()
    except AttributeError:
        course_desc_content = ""
        print("No content")
    
    if course_desc_content == "Chargement...":
        course_desc = course_desc_objective
    else:
        course_desc = course_desc_objective + " " + course_desc_content
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    if no_window_opened:
        driver.back()
        time.sleep(3)
        driver.find_element_by_id("tab-admission-et-plan-de-formation").click()
        time.sleep(1)
        driver.find_element_by_id("slider-plan-de-formation").click()
        time.sleep(1)
        driver.find_element_by_id("slider-profil-travail-etudes").click()
        time.sleep(1)
        no_window_opened = False
        
    else:    
        driver.close()
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)
        
print("Finished scraping {} courses".format(counter))

Scraped  GEN0009 Atelier : santé et sécurité au travail 
 Identifier les principales lois, normes et organismes reliés à la SST. Porter un jugement critique sur les structures en place. Contribuer à l'application des principales lois, règlements et normes dans son milieu de travail. Sensibiliser à l'impact des accidents de travail et à l'importance d'une législation dans ce domaine. Développer une approche préventive.
1 courses scraped

Scraped  GEN1005 Calcul I 
 Favoriser un arrimage harmonieux entre la formation mathématique acquise au collégial et les exigences de l'École. Comprendre l'importance, en regard des applications, de l'approximation de fonctions par des expressions n'impliquant que des opérations élémentaires (séries). Connaître les concepts fondamentaux relatifs aux fonctions à valeurs réelles mais qui dépendent de plusieurs variables réelles, ainsi qu'à leurs dérivées. Séries et approximations d'une fonction à une variable, polynôme et série de Taylor, erreur d'approxi

Scraped  GEN3115 Dessin et conception assistés par ordinateur 
 Être en mesure de visualiser en 3D et de représenter graphiquement des pièces et des assemblages de pièces mécaniques à l'aide des vues multiples et de perspectives standard. Vise à développer la capacité de communiquer et de s'exprimer graphiquement à l'aide de logiciels de dessin SolidWorks et Inventor. Les tolérances et les ajustements, l'environnement d'Inventor et de SolidWorks, l'affichage des menus appropriés à la tâche, les systèmes de coordonnées de l'utilisateur, l'ajustement des paramètres de dessin et de l'affichage des fenêtres de visualisation. La création et la gestion de calques, la construction, la modélisation et la modification de formes simples à complexes. Différentes stratégies pour augmenter la productivité : le dessin prototype, le regroupement d'entités, les échanges avec d'autres fichiers, etc. La sortie sur papier en respectant les conventions des communications graphiques : les vues, les formats

Scraped  GEN4230 Éléments d'électrotechnique et d'électronique 
 Donner des connaissances de base sur l’analyse de circuits électriques monophasés et triphasés et des circuits électroniques; faire connaître les particularités des circuits alternatifs et le fonctionnement des réseaux électriques avec les différentes machines utilisées dans l’industrie; faire connaître l’application des circuits d’électronique industrielle et de puissance. Concepts fondamentaux : résistance, inductance, capacité, sources dépendantes et indépendantes, sources contrôlées. Lois des mailles et des nœuds. Notions de phaseurs, d'impédances et de puissance. Circuits monophasés et triphasés. Circuits magnétiques, transformateurs, machines électriques. Distribution industrielle et utilisation de l’énergie électrique. Éléments d’électronique : diodes, transistors, thyristors, amplificateurs opérationnels. Introduction à l’électronique industrielle et de puissance. Variateurs électroniques de puissance.
21 courses 

Scraped  GEN4233 Dynamique des mécanismes complexes 
 Permettre à l'étudiant de s'initier aux principes de calcul des systèmes mécaniques complexes dans un contexte industriel. Structure des mécanismes : définitions, classifications, transformations des couples cinématiques, modules.  Mécanismes à membrures articulées : analyse cinématique des mécanismes plans, synthèse cinématique des mécanismes plans.  Cames : définitions, analyse cinématique, synthèse, optimisation des dimensions.  Cinétostatique : les forces en mécanismes, équilibrage dynamique.  Dynamique : bilan énergétique, rendement, modèles dynamiques, mouvement réel, réglage du mouvement.  Introduction à la robotique.  Conception assistée par ordinateur.  Applications industrielles.
30 courses scraped

Scraped  GEN0215 Analyse numérique 
 Acquérir des notions d'analyse numérique. Arithmétique des calculateurs. Analyse d'erreurs dans les opérations élémentaires. Chiffres significatifs exacts.  Équations non-linéaires, méthodes

Scraped  GEN1401 L'ingénieur et la société IV 
 Permettre à l'étudiant de se sensibiliser à ses devoirs envers sa profession, ses collègues et la société. Déontologie (suite). La loi et l'ingénieur; la pratique du génie, adhésion à l'Ordre des ingénieurs du Québec, etc.
41 courses scraped

Scraped  GEN4401 Conception en ingénierie II 
 Permettre à l'étudiant d'approfondir la méthodologie associée à la conception en ingénierie et l'appliquer à un problème concret d'envergure soumis de préférence par l'industrie. Rencontre hebdomadaire de 3 heures du professeur avec l'étudiant ou l'équipe d'étudiants pour le suivi et la validation du volet conception en ingénierie (design) lié au Projet appliqué de fin d'études.
42 courses scraped

Scraped  GEN4404 Projet appliqué de fin d'études 
 Développer chez l'étudiant l'esprit de synthèse et d'innovation dans le cadre d'un projet de conception en ingénierie.  Développer l'esprit du travail d'équipe et de leadership. Développer les techniques de re

Scraped  GEN4234 Fiabilité et maintenance 
 Apprendre la théorie de la fiabilité des équipements (machines) de production et pouvoir l'appliquer en milieu industriel. Apprendre la théorie et les techniques de gestion de la maintenance en entreprise manufacturière et pouvoir implanter un système de gestion de maintenance.

Familiarisation avec les principes de base de fiabilité, de disponibilité et de logistique de maintenance. Applications aux modélisations de fiabilité de systèmes complexes et aux développements et aux implantations de systèmes de maintenance préventive. Conception de système intégré de fiabilité ou d'un programme de maintenance dans une entreprise. Identification des lois de fiabilité. Optimisation des politiques de maintenance. Surveillance et diagnostic vibratoire. Maintenance et risques industriels.
51 courses scraped

Scraped  GEN4270 Énergétique 
 Donner aux étudiants des notions de base sur l’énergie et les technologies de la production d’énergie renouvelable e

# 4. Inspect and write to CSV

In [37]:
course_codes

['GEN0009',
 'GEN1005',
 'GEN1100',
 'GEN2210',
 'GEN2250',
 'GEN3307',
 'GEN4110',
 'GEN1007',
 'GEN1201',
 'GEN2800',
 'GEN3115',
 'GEN4139',
 'GEN5110',
 'MEC1400',
 'GEN1004',
 'GEN3250',
 'GEN3420',
 'GEN4129',
 'MEC2209',
 'GEN2813',
 'GEN4230',
 'GEN4301',
 'MEC2200',
 'MEC2410',
 'GEN0001',
 'GEN2192',
 'GEN3321',
 'GEN4122',
 'GEN4204',
 'GEN4233',
 'GEN0215',
 'GEN2913',
 'GEN3112',
 'GEN4402',
 'GEN4403',
 'GEN4460',
 'GEN0002',
 'GEN1302',
 'GEN2805',
 'GEN5124',
 'GEN1401',
 'GEN4401',
 'GEN4404',
 'GEN4112',
 'GEN4118',
 'GEN4135',
 'GEN4138',
 'GEN4140',
 'GEN4205',
 'GEN4213',
 'GEN4234',
 'GEN4270',
 'GEN4305',
 'GEN5220',
 'GEN4045',
 'GEN4047',
 'GEN4048',
 'GEN4137',
 'STI0501',
 'STI0502',
 'STI0503']

In [38]:
course_names

['Atelier : santé et sécurité au travail',
 'Calcul I',
 "Introduction au génie et aux projets d'ingénierie",
 'Statique',
 'Communication graphique',
 'Informatique I',
 'Chimie pour ingénieurs',
 'Calcul II',
 "L'ingénieur et la société II",
 'Morphologie des machines',
 'Dessin et conception assistés par ordinateur',
 'Techniques de mesure en génie mécanique',
 'Matériaux',
 'Résistance des matériaux I',
 'Équations différentielles ordinaires et aux dérivées partielles',
 'Probabilités et statistiques',
 'Logiciels de simulation',
 'Fabrication mécanique',
 'Thermodynamique',
 'Laboratoire mécanique I',
 "Éléments d'électrotechnique et d'électronique",
 'Conception de systèmes mécaniques',
 'Dynamique des fluides',
 "Dynamique de l'ingénieur",
 'Atelier : gestion et planification des projets I',
 'Analyse économique en ingénierie',
 'Systèmes asservis',
 'Systèmes hydrauliques et lubrification',
 'Théorie et conception des turbomachines',
 'Dynamique des mécanismes complexes',
 'Ana

In [39]:
course_descs

["Identifier les principales lois, normes et organismes reliés à la SST. Porter un jugement critique sur les structures en place. Contribuer à l'application des principales lois, règlements et normes dans son milieu de travail. Sensibiliser à l'impact des accidents de travail et à l'importance d'une législation dans ce domaine. Développer une approche préventive.",
 "Favoriser un arrimage harmonieux entre la formation mathématique acquise au collégial et les exigences de l'École. Comprendre l'importance, en regard des applications, de l'approximation de fonctions par des expressions n'impliquant que des opérations élémentaires (séries). Connaître les concepts fondamentaux relatifs aux fonctions à valeurs réelles mais qui dépendent de plusieurs variables réelles, ainsi qu'à leurs dérivées. Séries et approximations d'une fonction à une variable, polynôme et série de Taylor, erreur d'approximation, critères de convergence. Nombres complexes : arithmétique des nombres complexes, représenta

In [40]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,GEN0009,Atelier : santé et sécurité au travail,"Identifier les principales lois, normes et org..."
1,GEN1005,Calcul I,Favoriser un arrimage harmonieux entre la form...
2,GEN1100,Introduction au génie et aux projets d'ingénierie,Amener les étudiants à découvrir en quoi consi...
3,GEN2210,Statique,Rendre capable de calculer les composantes vec...
4,GEN2250,Communication graphique,Faire connaître les méthodes de communication ...
...,...,...,...
56,GEN4048,Sujets spéciaux IV,Permettre à l’étudiant d’acquérir des connaiss...
57,GEN4137,Mécanique du bâtiment,Acquérir des notions fondamentales de concepti...
58,STI0501,Stage industriel I,Permettre à l'étudiant de s'initier aux problé...
59,STI0502,Stage industriel II,Permettre à l'étudiant de s'initier aux problé...


In [41]:
df.to_csv('UQAT_GenieMecanique_Core_and_Electives_Courses.csv', index = False)

In [42]:
driver.quit()